In [1]:
import pandas as pd 
import eurostat
df = eurostat.get_data_df('sbs_sc_sca_r2', flags=False)
df

,nace_r2,indic_sb,size_emp,geo\time,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005
0,B,V11110,0-9,AT,NaN,183.0,192.0,215.0,226.0,230.0,227.0,238.0,238.0,242.0,245.0,244.0,237.0,NaN,NaN,NaN
1,B,V11110,0-9,BA,NaN,101.0,102.0,121.0,129.0,132.0,121.0,118.0,121.0,150.0,NaN,NaN,NaN,NaN,NaN,NaN
2,B,V11110,0-9,BE,NaN,140.0,118.0,173.0,144.0,167.0,149.0,175.0,160.0,227.0,249.0,248.0,NaN,NaN,NaN,NaN
3,B,V11110,0-9,BG,NaN,190.0,211.0,215.0,224.0,240.0,228.0,243.0,252.0,245.0,241.0,230.0,199.0,NaN,NaN,NaN
4,B,V11110,0-9,CH,NaN,82.0,84.0,81.0,85.0,84.0,86.0,89.0,75.0,75.0,79.0,85.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475052,S95,V92100,TOTAL,RS,NaN,4.4,4.3,4.2,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475053,S95,V92100,TOTAL,SE,NaN,1.1,1.1,1.2,1.2,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475054,S95,V92100,TOTAL,SI,NaN,1.5,1.5,1.5,1.5,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475055,S95,V92100,TOTAL,SK,NaN,1.5,1.4,1.5,1.4,1.4,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#eliminate columns for years previous to 2011
years =[]
for y in range(2005,2011): 
    years.append(y)
df=df.drop(years, axis=1)#drop columns with years before 2011
#rename geo time column to be easier to handle because of the name
df.rename(columns={"geo\\time": "geo"}, inplace=True)

In [3]:
#FUNCTION CALLABLE TO SELECT FOR EXAMPLE YEARS AND VARIABLE
import PySimpleGUI as sg
def selection(option):
    #sg.theme('DarkAmber')   # Add a touch of color
    #select the variable we want to calculate
    options = option

    # All the stuff inside your window.
    layout = [ 
            [sg.Text('Select one->'), sg.Listbox(options,select_mode=sg.LISTBOX_SELECT_MODE_SINGLE,size=(20,len(options)))],
            [sg.Button('Ok'), sg.Button('Cancel')]
        ]

    # Create the Window
    window = sg.Window('Make your choice', layout)

    # Event Loop to process "events" and get the "values" of the input
    while True:
        event, values = window.read()
        print( f"event={event}" )
        if event is None or event == 'Ok' or event == 'Cancel': # if user closes window or clicks cancel
            break
        
    # close  the window        
    window.close()

    if event == "Cancel":
        print( "You cancelled" )
    else:
        print('You entered ', values[0])
        sg.popup( f"You selected {values[0]}" )        
    return values

In [4]:
#selection of the variable (Calling the function)
dic_indic_sb = eurostat.get_dic('indic_sb') #dictionary for all the indicators in sbs
for indic in df.indic_sb.unique():
    if indic in dic_indic_sb:
        print (indic + " -"+dic_indic_sb[indic])

var=selection(df.indic_sb.unique())[0]
#determine the variable of interest based on the selection
for indic in var:
    if indic in dic_indic_sb:
        variable_of_interest=dic_indic_sb[indic]

print(variable_of_interest)
print(" ")


#create a list of years for selection
yearsloop =[]
for y in range(2011,2021): 
    yearsloop.append(y)
#select de year frame (Calling the function) with the starting year and the end year
year_start=selection(yearsloop)[0] #with [] we convert variable to a list to apply "isin"
year_end=selection(yearsloop)[0]
print(year_start)
print(year_end)

V11110 -Enterprises - number
V12110 -Turnover or gross premiums written - million euro
V12120 -Production value - million euro
V12150 -Value added at factor cost - million euro
V12170 -Gross operating surplus - million euro
V13110 -Total purchases of goods and services - million euro
V13310 -Personnel costs - million euro
V13320 -Wages and Salaries - million euro
V13330 -Social security costs - million euro
V16110 -Persons employed - number
V16120 -Unpaid persons employed - number
V16130 -Employees - number
V16150 -Hours worked by employees - number
V91100 -Turnover per person employed - thousand euro
V91110 -Apparent labour productivity (Gross value added per person employed) - thousand euro
V91120 -Wage adjusted labour productivity (Apparent labour productivity by average personnel costs) - percentage
V91130 -Gross value added per employee - thousand euro
V91150 -Gross value added per hour worked by employees
V91170 -Share of personnel costs in production - percentage
V91210 -Average

In [5]:
#select nace
nace_sel=['C31', 'C32', 'N80','N81','N82']

#countries to eliminate
countries_not_selected= ['UK', 'TR', 'AL','IS','LI','NO','CH','UK','ME','MK','AL','RS','TR','BA','EU28','EU15','EA','EA19','EA12', 'EU27_2007']
# columns of countries to be eliminated after sum
cols = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI','FR', 'HR', 'HU', 'IE', 'IT', 'LT','LU', 'LV', 'MT','NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']

#size total values (no distribution by size)
size_sel=['TOTAL']#we select the totals

In [6]:
import numpy as np
results_tot = pd.DataFrame() #create an empty dataframe for total results
#filter by size, nace, variable and countries
df_selection=df[df.size_emp.isin(size_sel)&df.nace_r2.isin(nace_sel)&df.indic_sb.isin(var)&~df.geo.isin(countries_not_selected)]
df_selection

,nace_r2,indic_sb,size_emp,geo,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
312990,C31,V16110,TOTAL,AT,26260.0,27209.0,27547.0,27218.0,26945.0,27677.0,27975.0,28463.0,29492.0,29830.0
312992,C31,V16110,TOTAL,BE,10463.0,13166.0,12366.0,12079.0,12255.0,12506.0,12923.0,12898.0,13004.0,14414.0
312993,C31,V16110,TOTAL,BG,20437.0,23474.0,23839.0,22730.0,22687.0,21839.0,21938.0,21347.0,21897.0,22119.0
312995,C31,V16110,TOTAL,CY,1035.0,1081.0,1033.0,952.0,889.0,856.0,892.0,1006.0,1136.0,1300.0
312996,C31,V16110,TOTAL,CZ,23658.0,24613.0,24931.0,25760.0,25986.0,26141.0,25972.0,26299.0,28016.0,28497.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472600,N82,V16110,TOTAL,PT,238102.0,233508.0,224133.0,217644.0,200887.0,188625.0,176497.0,164139.0,162544.0,170027.0
472601,N82,V16110,TOTAL,RO,74237.0,72946.0,70085.0,67217.0,62064.0,58437.0,53529.0,51425.0,45375.0,41579.0
472603,N82,V16110,TOTAL,SE,56660.0,56476.0,53628.0,49420.0,48039.0,51678.0,53111.0,53649.0,49029.0,47996.0
472604,N82,V16110,TOTAL,SI,4818.0,4606.0,4726.0,4328.0,3703.0,3212.0,2880.0,2646.0,2590.0,2551.0


In [7]:
#iteration to retrieve the data for each year
for i in range (year_start[0],year_end[0]+1):
    results = pd.DataFrame() #create an empty dataframe for partial results
    df_target = pd.pivot_table(df_selection, values=i, index=['nace_r2'],columns=["geo"], aggfunc=np.sum, fill_value=0)#rearrange using nace
    df_target['sum'] = df_target[cols].sum(axis=1)#sums the columns of the countries to "sum"
    df_target=df_target.drop(cols, axis=1)#eliminate countries columns
    #now we can locate columns with 0 and make the substitution with the column "sum"
    df_target.loc[df_target['EU27_2020']==0,'EU27_2020'] = df_target['sum']#adds summ if column is empty
    moddf_target=df_target.drop('sum', axis=1)#drop sum column
    column_name=str(variable_of_interest)+str(i)#determine de column name using variable of interest and year
    results[column_name]= moddf_target['EU27_2020']
    results_tot = pd.concat([results_tot,results], axis=1)#adds column in each iteration (for each year)
results_tot

,Persons employed - number2011,Persons employed - number2012,Persons employed - number2013,Persons employed - number2014,Persons employed - number2015,Persons employed - number2016,Persons employed - number2017,Persons employed - number2018,Persons employed - number2019
nace_r2,,,,,,,,,
C31,955314,913531,889265,887186,900000,900000,928438,950000,1000000
C32,800000,780000,790000,809098,809182,817907,851795,900000,900000
N80,1200000,1213443,1200000,1190000,1200000,1290000,1280000,1290000,1300000
N81,3644726,3663453,3743508,3882333,3936929,4194979,4318543,4517207,4627454
N82,1740000,1800000,1840000,1949889,2058609,2150000,2234338,2243654,2336951


In [8]:
results_tot=results_tot.astype(float) #as originally data is integer need to change to float to compute division
results_tot.dtypes


Persons employed - number2011    float64
Persons employed - number2012    float64
Persons employed - number2013    float64
Persons employed - number2014    float64
Persons employed - number2015    float64
Persons employed - number2016    float64
Persons employed - number2017    float64
Persons employed - number2018    float64
Persons employed - number2019    float64
dtype: object

In [9]:
#create a dataframe for the ratios (copying the result_tot)
ratio_results =  results_tot.copy()
years= year_end[0]-year_start[0]
for i in range (0,years+1):
    ratio_results.iloc[0][i]=results_tot.iloc[0][i]/(results_tot.iloc[0][i]+results_tot.iloc[1][i])
    ratio_results.iloc[1][i]=results_tot.iloc[1][i]/(results_tot.iloc[0][i]+results_tot.iloc[1][i])
    ratio_results.iloc[2][i]=results_tot.iloc[2][i]/(results_tot.iloc[2][i]+results_tot.iloc[3][i]+results_tot.iloc[4][i])
    ratio_results.iloc[3][i]=results_tot.iloc[3][i]/(results_tot.iloc[2][i]+results_tot.iloc[3][i]+results_tot.iloc[4][i])
    ratio_results.iloc[4][i]=results_tot.iloc[4][i]/(results_tot.iloc[2][i]+results_tot.iloc[3][i]+results_tot.iloc[4][i])
    #_tot.iloc[0][1]#=ratio_tot.iloc[0][i](ratio_tot.iloc[0][i]+ratio_tot.iloc[1][i])
ratio_results    
    #=ratio_tot.iloc["C31"][i]/(ratio_tot.iloc["C31", i]+ratio_tot.iloc["C32", i])


,Persons employed - number2011,Persons employed - number2012,Persons employed - number2013,Persons employed - number2014,Persons employed - number2015,Persons employed - number2016,Persons employed - number2017,Persons employed - number2018,Persons employed - number2019
nace_r2,,,,,,,,,
C31,0.544241,0.539424,0.529556,0.523017,0.526568,0.523893,0.521526,0.513514,0.526316
C32,0.455759,0.460576,0.470444,0.476983,0.473432,0.476107,0.478474,0.486486,0.473684
N80,0.182240,0.181738,0.176900,0.169462,0.166770,0.168959,0.163414,0.160231,0.157301
N81,0.553512,0.548676,0.551854,0.552864,0.547135,0.549442,0.551335,0.561084,0.559926
N82,0.264248,0.269586,0.271246,0.277674,0.286095,0.281599,0.285251,0.278685,0.282773


In [10]:
#save to file to calculate ration
fileratio="ratioCN-"+ variable_of_interest
filename = "%s.xlsx" % fileratio
ratio_results.to_excel(filename)